In [1]:
from planet4 import markings, dbscan, io, fnotching

In [6]:
from nbtools.logging import setup_live_logging
import logging

In [11]:
setup_live_logging('planet4', logging.DEBUG)

<Logger planet4 (DEBUG)>

# Data to cluster

In [7]:
from planet4 import region_data

In [3]:
%store -r

In [13]:
len(combined)

125

In [2]:
db = io.DBManager()

In [4]:
db.dbname

'/Users/klay6683/Dropbox/data/planet4/2017-01-01_planet_four_classifications_queryable_cleaned_seasons2and3.h5'

In [5]:
obsids = db.image_names

In [6]:
len(obsids)

221

# Support functions

In [9]:
def do_cluster_obsids(obsid, savedir=None):
    from planet4 import dbscan, fnotching
    dbscanner = dbscan.DBScanner(savedir=savedir)
    dbscanner.cluster_image_name(obsid)
    fnotching.fnotch_image_ids(obsid, savedir=savedir)
    fnotching.apply_cut(obsid, savedir=savedir)
    return obsid

In [9]:
def do_cluster_ids(imgid, savedir=None):
    from planet4 import dbscan, fnotching
    dbscanner = dbscan.DBScanner(savedir=savedir)
    dbscanner.cluster_image_id(imgid)
    obsid = dbscanner.pm.obsid
    fnotching.fnotch_image_ids(obsid, savedir=savedir,
                               scope='planet4')
    fnotching.apply_cut(obsid, savedir=savedir)
    return imgid

In [10]:
def do_only_fnotching(obsid, savedir=None):
    fnotching.fnotch_image_ids(obsid, savedir=savedir)
    fnotching.apply_cut(obsid, savedir=savedir)

In [11]:
for imgid in image_ids:
    do_cluster_ids(imgid, "without_large_fan_fnotch_on_p4_coords" )

'APF0000nfw'

'APF0000nbo'

'APF0000nel'

'APF0000nee'

In [16]:
obsid = 'ESP_020930_0980'
savedir = 'fnotching_per_obsid'

In [12]:
dbscanner = dbscan.DBScanner(savedir=savedir)

In [13]:
dbscanner.cluster_image_name(obsid)

In [46]:
pm.final_blotchfile

PosixPath('/Users/klay6683/Dropbox/data/planet4/fnotching_per_obsid/ESP_020930_0980/L1C_cut_0.5/L1C_cut_0.5__blotches.csv')

In [40]:
fnotching.fnotch_obsid(obsid, savedir=savedir)

In [47]:
fnotching.apply_cut_obsid(obsid, savedir=savedir)

In [67]:
data = region_data.Inca.season2 + region_data.Inca.season3

In [32]:
do_clustering(ithaca[0])

'ESP_011350_0945'

In [33]:
ithaca[0]

'ESP_011350_0945'

In [34]:
from nbtools import execute_in_parallel

In [68]:
result = execute_in_parallel(do_clustering, data)

# Creating obsid files

In [61]:
from planet4.fnotching import get_clusters_in_path

In [62]:
def create_obsid_result_files(obsid, datapath=None):
#     logger.debug("Working on %s", obsid)
    pm = io.PathManager(obsid=obsid, datapath=datapath)
    # the pm.get_obsid_paths method searches for existing image_id folders inside each
    # obsid folder
    folders = list(pm.get_obsid_paths(pm.L1C_folder))
    blotches = []
    fans = []
    for folder in folders:
        pm.id = fnotching.get_id_from_path(folder)
        if pm.final_blotchfile.exists() and pm.final_blotchfile.lstat().st_size > 1:
            blotches.append(pm.final_blotchdf)
        if pm.final_fanfile.exists() and pm.final_fanfile.lstat().st_size > 1:
            fans.append(pm.final_fandf)

    if len(blotches) > 0:
        blotches = pd.concat(blotches, ignore_index=True)
        blotches.dropna(how='all', axis=1, inplace=True)
        blotches.to_csv(pm.obsid_final_blotches_path, index=False)
        print("Created", pm.obsid_final_blotches_path)

    if len(fans) > 0:
        fans = pd.concat(fans, ignore_index=True)
        fans.dropna(how='all', axis=1, inplace=True)
        fans.to_csv(pm.obsid_final_fans_path, index=False)
        print("Created", pm.obsid_final_fans_path)

In [69]:
for obsid in data:
    create_obsid_result_files(obsid, datapath='for_chase')

Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_013113_0985_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_013113_0985_fans.csv
Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_013034_0985_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_013034_0985_fans.csv
Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_012889_0985_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_012889_0985_fans.csv
Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_012744_0985_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_012744_0985_fans.csv
Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_012691_0985_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_012691_0985_fans.csv
Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_012467_0985_blotches.csv
Created /Users/klay6683/Dropbox/data/planet4/for_chase/ESP_012467_0985_fans.csv
Created /Users/k

In [17]:
name= "Ithaca"
region = getattr(region_data, name)
for season_no in [2, 3]:
    season = str(season_no)
    bucket = []
    for obsid in getattr(region, f"season{season}"):
        pm = io.PathManager(obsid=obsid)
        df = pd.read_csv(pm.obsid_final_blotches_path)
        df['obsid'] = obsid
        bucket.append(df)

    df = pd.concat(bucket, ignore_index=True)
    df.to_csv(io.analysis_folder() / f"p4_catalog/{name}_season{season}_blotches.csv",
              index=False)

In [47]:
store_folder = io.analysis_folder() / 'for_chase'
store_folder.mkdir(exist_ok=True)

In [70]:
name = 'inca'

In [71]:
p = Path('/Users/klay6683/Dropbox/data/planet4/for_chase/' + name)
for kind in ['blotches', 'fans']:
    bucket = []
    filepaths = p.glob(f'*_{kind}.csv')
    for filepath in filepaths:
        obsid = filepath.name[:15]
        df = pd.read_csv(filepath)
        df['obsid'] = obsid
        bucket.append(df)
    df = pd.concat(bucket, ignore_index=True)
    savedir = store_folder / f'{name}'
    savedir.mkdir(exist_ok=True)
    savepath = savedir / f"{name}_{kind}.csv"
    print(savepath)
    df.to_csv(savepath, index=False)
        

/Users/klay6683/Dropbox/data/planet4/p4_analysis/for_chase/inca/inca_blotches.csv
/Users/klay6683/Dropbox/data/planet4/p4_analysis/for_chase/inca/inca_fans.csv
